## Section 10: The world's simplest model, now simpler

* Intercept-only model in Bambi
* Quick review of the model math
    * Just a slide to review the concept
* Create the model in Bambi
    * `model = bmb.Model("weight ~ 1", data)`
    * We aren't passing priors explicitly. We'll come back to this in the next section.
* Fit the model in Bambi
    * `idata = model.fit()`
    * Perhaps pass kwargs to show it
* Explain what we just done
    * `Model` class is our entry point to create all models in Bambi
    * The model formula is the concise description of predictors that Bambi used
    * The tilde "~" separates the response from the predictors
    * The `1` stands for the intercept.
        * It's as if the slope was multiplied by a constant value, which is 1.

### Discussion points

* Should we print the model here, and explain what are the components?
    * This would make us explain at least a little about the priors or live with the confusion.
    * I think that would mean two big concepts into the same section, and we don't want that

## Section 20: What's in a Bambi model

* What happens when we print a Bambi model?
    * So much information is made available to us!
    * Formula -> The one we entered
    * Family -> It's a broader concept, but for now we're happy to see it says "gaussian".
        * It means we're using a Gaussian likelihood function
    * Link -> It's an important concept that will be very well covered in Lesson 4.
        * In a nutshell, it's a transformation that is applied to the mean of the likelihood function
        * For now we're happy knowing it says "identity" which means it doesn't change anything
    * Observations: Helpful to double check!
    * And then we have priors! 
        * Common-level effects show the priors for predictor related parameters, like the slopes or the intercept!
        * Auxiliary parameters are all the parameters that go into the likelihood, but are not affected by predictors
            * Like the sigma in the linear regression model!
        * How Bambi chooses priors is another topic.
            * But it tries to do it in such a way they are non-informative
* And where's the PyMC model?
    * You can access it with `model.backend.model`
    * It's not advisable to work with it directly, but if you can do it if you just want to have some fun
* Other cool tricks
    * .plot_prior()
    * .prior_predictive()
        * I wouldn't show this one
    * .graph()

### Discussion points

* Are the following two "sections" needed?
    * "And where's the PyMC model?"
    * "Other cool tricks"

## Section 30: Slopes

* Adding a slope is so easy
    * Just add the name of the predictor to the model formula
    * "y ~ 1 + response"
    * Even if it has a transformation!
        * We can use functions in the model formula, such as `np.log(x)`
        * It's allowed for both the response and the predictors
* Show model description, see the automatic prior
* Fit model, see everything looks well
    * Some visualizations as well

### Discussion points

* Show plot_cap?
* I'm afraid this may be too short lesson... but is it a problem?
* We could discuss more about tranformations here, but I'm not sure if it's the right place
    * Again, it would put two important topics (add a slope and how tranformation works) in one section.
    * I think it's better to have it as the next section.

## Section 40: Transformations in Bambi

* How is it that `np.log(x)` worked?
    * Is it because Bambi loads numpy internally?
        * Nope. It's because Bambi allows you to use functions available in the environment where you create the Bambi model
        * Plus, it has built-in transformations!
* Built-in transformations
    * The most important are
        * Center
        * Scale
    * Show what they do, and when they are recommended
* Extra message:
    * If you are going to use centered or scaled predictors, it is better to let Bambi handle it with Center or Scale rather than doing it manually in the data frame. There are some rough edges that Bambi will make sure you avoid.

### Discussion points

* Is this section really necessary?
    * I think it's a nice to have
    * But it could be an appendix as well

### Section 50: Modeling categories

* Even though species is of a different nature, we don't need to change anything to the model formula
    * We just pass it, and Bambi knows how to handle it
* "y ~ 1 + species"
    * Why does Bambi drop the first species in species? 
        * Because the species variable would be confounded with the intercept.
        * And that would imply the model is non-identifiable.
        * Dropping a level is one of the many constrains one could apply to the categorical variable to make the model identifiable.
* "y ~ 0 + species" 
    * But another option is dropping the intercept
    * Since there's no intercept, there's no need to apply any restriction to the categorical predictor and Bambi retains a coefficient for every species. Just like the PyMC model we wrote in the exercises of Lesson 2.
    * So here we're also showing the meaning of the "0" in the model formula.
* Show the estimated coefficients are different under both models...
    * Finish the section showing this, and wondering why this is happening
    * Say this will be covered in the next section.


### Discussion points

* It would be possible to add the explanations to the different encodings here, but I think it would make the section too long and complex.

## Section 60: Understanding encodings

* Categorical variables need to be encoded into numbers somehow
* A default approach is to create dummy variables
* But since the sum(x, axis=1) adds up to a column of 1s, keeping all the dummies would introduce perfect association between predictors if we also have an intercept
    * And for complicated reasons this is not something we want. 
    * It's going to break our inferences
    * We say the model is not-identified
* Different alternatives
    * Never use an intercept
        * It's successful only if we have one categorical predictor
        * The meaning of the coefficient is the mean of the group
            * This is known as cell-means encoding
    * Use encodings that apply restrictions
        * By default, Bambi uses Treatment encoding (or Reference encoding)
        * One level is dropped, and that level is the Reference level of the variable
        * The meaning of the coefficients is then the difference between the any category and the reference level
* It's a complex topic
    * There are many different encodingss
    * And things get more complicated when there are more categorical terms, it's hard to make sure we're not introducing non-identifiabilities
    * Fortunately, Bambi handles all these cases automatically for us
    * Even though it's not completely clear for us, we can rely on Bambi :)
    * But it's always good to try to figure out what's going on!!!

## Section 70: The full model

Goals: 

* Show how to create the model with multiple intercepts and multiple slopes
* We need to introduce the interaction operator
    * `numeric:categoric` gives different slopes of `numeric` for every level in `categoric`

## Section 80: Predictions

* .predict() method
* much simpler than PyMC
* Out-of-sample predictions is _extremely_ easy

## Section 90: End to end analysis with Bambi

* Model specification
* Explore priors
* Explore prior predictive distribution
* Change priors?
* Updte model
* Explore model again
* Fit model
* Explore posteriors
* Diagnose posteriors
* Show fitted curve
* Predict weigth for new observations
* ... I think this is it

## To discuss

* When do we show how to change the automatic priors?
    * Is it needed in this lesson?
        * I feel yes, but I'm not sure what's the right place
* Predictive distributions
    * Students should be familiar because of the introductory course.
    * I think we _could_ show them without much explanation, but it would be better to provide one or two sentences.

### Concepts and things I don't want to miss

* Bambi
* Automatic Priors
* Formulae syntax
* Prior predictive sampling in Bambi
* Predictions in Bambi

...

* Bambi has automatic priors.
  * Main points:
    * Designed to provide low to moderate shrinkage/regularization in most situations.
    * Allows faster prototyping and iteration.
    * Helps you getting focused on the model and the problem. You can refine priors once your model is selected.
* Bambi allows to pass inline transforms (eg log(y) ~ x).
    * Compare manual transform with inline transform.
* Show Bambi repr and how nice it summarizes everything

... 

* At the end of this lesson this person should feel impressed with Bambi and understand
  * The equivalent functionaly
  * Syntatical sugar of Bambi
  * Less code to type, fewer errors to make.
  * Bambi allows to iterate faster when exploring different models.

...


* Bambi vs PyMC comparison (in terms of code)
* When do we talk about power terms?
    * They will be useful for the concrete example